In [1]:
import numpy as np
import random

In [2]:
class Network(object):
    
    # Initializing method
    def __init__(self, sizes):
        self.num_layers = len(sizes) # getting number of layers
        self.sizes = sizes # number of neurons in each layers
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]  # assign bias
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])] # assign random weights
    
    # Method for feedforward
    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    #stochastic gradient descent
    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        if test_data: # If test data is present then assign n_test with the size of test data
            n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data) # shuffle the training data
            mini_batches = [training_data [k: k + mini_batch_size] #taking samples from the training data if size mini_batch size
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
#updates the network weights and biases according to a single iteration of gradient descent, using just the training data in mini_batch
                self.update_mini_batch(mini_batch, eta) #eta is the learning rate
            if test_data:                                  # test data present then evaluate the accuracy
                print ("Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test))
            else:
                print ("Epoch {0} complete".format(j))
                
    # update the mini batches created above
    """Update the network's weights and biases by applying
    gradient descent using backpropagation to a single mini batch.
    The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
    is the learning rate."""
    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw 
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb 
                       for b, nb in zip(self.biases, nabla_b)]
        
    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)
    
# Derivative of sigmoid function
def sigmoid_prime(z):
        return sigmoid(z)*(1-sigmoid(z))
    
# Sigmoid used as activation function function
def sigmoid(z):
        """The sigmoid function."""
        return 1.0/(1.0+np.exp(-z))

In [3]:
"""
mnist_loader
~~~~~~~~~~~~

A library to load the MNIST image data.  For details of the data
structures that are returned, see the doc strings for ``load_data``
and ``load_data_wrapper``.  In practice, ``load_data_wrapper`` is the
function usually called by our neural network code.
"""

#### Libraries
# Standard library
import pickle
import gzip


def load_data():
    """Return the MNIST data as a tuple containing the training data,
    the validation data, and the test data.

    The ``training_data`` is returned as a tuple with two entries.
    The first entry contains the actual training images.  This is a
    numpy ndarray with 50,000 entries.  Each entry is, in turn, a
    numpy ndarray with 784 values, representing the 28 * 28 = 784
    pixels in a single MNIST image.

    The second entry in the ``training_data`` tuple is a numpy ndarray
    containing 50,000 entries.  Those entries are just the digit
    values (0...9) for the corresponding images contained in the first
    entry of the tuple.

    The ``validation_data`` and ``test_data`` are similar, except
    each contains only 10,000 images.

    This is a nice data format, but for use in neural networks it's
    helpful to modify the format of the ``training_data`` a little.
    That's done in the wrapper function ``load_data_wrapper()``, see
    below.
    """
    f = gzip.open('data/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding='latin1')
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.

    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.

    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.

    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code."""
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = list(zip(training_inputs, training_results))
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = list(zip(validation_inputs, va_d[1]))
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = list(zip(test_inputs, te_d[1]))
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [4]:
data = load_data_wrapper()
train = data[0]
test = data[2]

## Case 1 - Accuracy = 95.12%

In [5]:
net = Network([784, 30, 10]) # neurons to input layer is 784 neurons to hidden layer is 30 and output layer has 10 neurons
net.SGD(train, 30, 10, 3.0, test_data= test) # epoch 30 batch size 10 and learning rate as 3

Epoch 0: 9135 / 10000
Epoch 1: 9219 / 10000
Epoch 2: 9313 / 10000
Epoch 3: 9331 / 10000
Epoch 4: 9368 / 10000
Epoch 5: 9369 / 10000
Epoch 6: 9410 / 10000
Epoch 7: 9408 / 10000
Epoch 8: 9413 / 10000
Epoch 9: 9449 / 10000
Epoch 10: 9457 / 10000
Epoch 11: 9466 / 10000
Epoch 12: 9458 / 10000
Epoch 13: 9506 / 10000
Epoch 14: 9479 / 10000
Epoch 15: 9464 / 10000
Epoch 16: 9483 / 10000
Epoch 17: 9485 / 10000
Epoch 18: 9492 / 10000
Epoch 19: 9480 / 10000
Epoch 20: 9480 / 10000
Epoch 21: 9503 / 10000
Epoch 22: 9499 / 10000
Epoch 23: 9512 / 10000
Epoch 24: 9493 / 10000
Epoch 25: 9494 / 10000
Epoch 26: 9512 / 10000
Epoch 27: 9502 / 10000
Epoch 28: 9511 / 10000
Epoch 29: 9488 / 10000


## Case 2 - Accuracy = 68.95%

In [6]:
net = Network([784, 100, 10]) # neurons to input layer is 784 neurons to hidden layer is 100 and output layer has 10 neurons
net.SGD(train, 30, 10, 3.0, test_data=test)# epoch 30 batch size 10 and learning rate as 3

Epoch 0: 4006 / 10000
Epoch 1: 4990 / 10000
Epoch 2: 5813 / 10000
Epoch 3: 5983 / 10000
Epoch 4: 6746 / 10000
Epoch 5: 6754 / 10000
Epoch 6: 6778 / 10000
Epoch 7: 6813 / 10000
Epoch 8: 6817 / 10000
Epoch 9: 6831 / 10000
Epoch 10: 6822 / 10000
Epoch 11: 6842 / 10000
Epoch 12: 6841 / 10000
Epoch 13: 6849 / 10000
Epoch 14: 6865 / 10000
Epoch 15: 6860 / 10000
Epoch 16: 6859 / 10000
Epoch 17: 6856 / 10000
Epoch 18: 6863 / 10000
Epoch 19: 6868 / 10000
Epoch 20: 6869 / 10000
Epoch 21: 6873 / 10000
Epoch 22: 6874 / 10000
Epoch 23: 6882 / 10000
Epoch 24: 6891 / 10000
Epoch 25: 6883 / 10000
Epoch 26: 6886 / 10000
Epoch 27: 6879 / 10000
Epoch 28: 6896 / 10000
Epoch 29: 6895 / 10000


## Case 3 - Accuracy = 30.83% 

In [8]:
net = Network([784, 100, 10]) # neurons to input layer is 784 neurons to hidden layer is 100 and output layer has 10 neurons
net.SGD(train, 30, 10, 0.001, test_data=test) # epoch 30 batch size 10 and learning rate as 0.001

Epoch 0: 1027 / 10000
Epoch 1: 1078 / 10000
Epoch 2: 1472 / 10000
Epoch 3: 1854 / 10000
Epoch 4: 1982 / 10000
Epoch 5: 2062 / 10000
Epoch 6: 2138 / 10000
Epoch 7: 2199 / 10000
Epoch 8: 2253 / 10000
Epoch 9: 2306 / 10000
Epoch 10: 2367 / 10000
Epoch 11: 2406 / 10000
Epoch 12: 2453 / 10000
Epoch 13: 2490 / 10000
Epoch 14: 2543 / 10000
Epoch 15: 2578 / 10000
Epoch 16: 2627 / 10000
Epoch 17: 2661 / 10000
Epoch 18: 2689 / 10000
Epoch 19: 2719 / 10000
Epoch 20: 2744 / 10000
Epoch 21: 2779 / 10000
Epoch 22: 2802 / 10000
Epoch 23: 2831 / 10000
Epoch 24: 2877 / 10000
Epoch 25: 2923 / 10000
Epoch 26: 2953 / 10000
Epoch 27: 3003 / 10000
Epoch 28: 3048 / 10000
Epoch 29: 3083 / 10000


## Case 4 - Accuracy = 11.89%

In [9]:
net = Network([784, 30, 10])# neurons to input layer is 784 neurons to hidden layer is 30 and output layer has 10 neurons
net.SGD(train, 30, 10, 100.0, test_data=test)  # epoch 30 batch size 10 and learning rate as 100

Epoch 0: 962 / 10000
Epoch 1: 1164 / 10000
Epoch 2: 1152 / 10000
Epoch 3: 1154 / 10000
Epoch 4: 1153 / 10000
Epoch 5: 1152 / 10000
Epoch 6: 1143 / 10000
Epoch 7: 1139 / 10000
Epoch 8: 1189 / 10000
Epoch 9: 1184 / 10000
Epoch 10: 1174 / 10000
Epoch 11: 1171 / 10000
Epoch 12: 1163 / 10000
Epoch 13: 1146 / 10000
Epoch 14: 1018 / 10000
Epoch 15: 1018 / 10000
Epoch 16: 1018 / 10000
Epoch 17: 1018 / 10000
Epoch 18: 1017 / 10000
Epoch 19: 1017 / 10000
Epoch 20: 1016 / 10000
Epoch 21: 1016 / 10000
Epoch 22: 1016 / 10000
Epoch 23: 1016 / 10000
Epoch 24: 1016 / 10000
Epoch 25: 1016 / 10000
Epoch 26: 1016 / 10000
Epoch 27: 1016 / 10000
Epoch 28: 1016 / 10000
Epoch 29: 1016 / 10000
